# Section 1. Introduction to the problem/task and dataset

The group has selected the house dataset based on the list of datasets.

The target task of this notebook to predict the house prices in King County given selected house features. Using these models may be of interest to buyers or sellers who may not be familiar with the house market in King County.

Interested buyers for houses in King County may want to estimate their budget given their desired house features.

On the other hand, sellers who may not be familiar with the market may not want to underestimate or overestimate their house's worth. These models may be of assistance in guiding sellers to price their houses.

# Section 2. Description of the dataset

This dataset consists of house sale prices and sold houses between May 2014 and May 2015 in King County.

Each row represents a house sold and each column represents a feature of a house.
This dataset contains 21613 instances and 21 features overall.

Features:
- `id` – A notation for a house x
- `date` – Date sold x
- `price` – Sale price
- `bedrooms` – Number of bedrooms
- `bathrooms` – Number of bathrooms
- `sqft_living` – Size of living area in square feet
- `sqft_lot` – Size of the lot in square feet
- `floors` – Total floors in the house
- `waterfront` – ‘1’ if the property has a waterfront, ‘0’ if not.
- `view` – An index from 0 to 4 of how good the view of the property was.
- `condition` – Condition of the house, ranked from 1 to 5
- `grade` – Classification by construction quality which refers to the types of materials used
and the quality of workmanship. Buildings of better quality (higher grade) cost more to
build per unit of measure and command higher value.
- `sqft_above` –  Square feet above ground (find just to remove)
- `sqft_basement` – Square feet below ground (find just to bin)
- `yr_built` – Year built
- `yr_renovated` – Year renovated. ‘0’ if never renovated
- `zipcode` – 5-digit zip code (transform)
- `lat` – Latitude coordinate (transform)
- `long` – Longitude coordinate (transform)
- `sqft_living15` – Average size of interior housing living space for the closest 15 houses, in
square feet
- `sqft_lot15` – Average size of land lots for the closest 15 houses, in square feet

# Section 3. List of requirements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import ScalarFormatter

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Exploratory data analysis
from ydata_profiling import ProfileReport

# Geographical analysis
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from sklearn.cluster import KMeans

# extracting lat, long from zipcode
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# for google Colab
# from google.colab import drive

# Model & Model training/performance
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

import torch.nn as nn
import torch.optim as optim
import torch.nn.init
import seaborn as sns

# documentation
from IPython.display import Image

# Section 4. Data preprocessing and cleaning

The csv will be first loaded. Displaying the head for a quick look at the data.

In [ ]:
house_df = pd.read_csv("house_prices.csv")
house_df.head()

Let's also take a look into the data types that pandas set for the dataset

In [ ]:
house_df.info()

## Duplicate data


In [ ]:
for col in house_df.columns:
    print(f'{col}:\t\t{house_df['bathrooms'].is_unique}')

Other than ID, all the other parameters are expected to be non-unique so we investigate further into duplicated ID values. 

In [ ]:
a = house_df["id"].value_counts()
dupe_id_df = house_df.join(a, on='id')
dupe_id_df[dupe_id_df['count'] > 1]

A total of 353 are rows that contain a repeated ID value. It is observed that duplicate IDs are from house prices that change overtime. An example is shown below. 

In [ ]:
house_df[house_df.id == 795000620]

Since our model assumes the current price of the house and not its price overtime or on a specific date, we decided that we don't need to feed the previous price of the houses. Therefore, we are only keeping the newest row.

In [ ]:
# keeping only most recent ids for dupes
house_df = house_df.drop_duplicates(subset=['id'], keep='last')
house_df.id.duplicated().value_counts()

This line of code should return the the row that has a year of 2015.

In [ ]:
house_df[house_df.id == 795000620]

In [ ]:
house_df.loc[house_df["id"] == 7129300520]

## Converting date into DateTime object
For us to be able to do more exploratory data analysis (EDA) and generate date related columns, we convert the date column into a `datetime` data type.


In [ ]:
# Convert date for easier processing
house_df['date'] = pd.to_datetime(house_df['date'])
house_df['date'].info()

## Check for the minimum and maximum values

The dataset may include rows with suspicious values so we investigate the maximum and minimum values for each column. 

In [ ]:
columns_to_drop = ['id', 'date', 'lat', 'long']
minMax_df = house_df.drop(columns=columns_to_drop)

min_values = minMax_df.min()
max_values = minMax_df.max()

min_max_df = pd.DataFrame({'min': min_values, 'max': max_values})
min_max_df = min_max_df.round(2)

print(min_max_df)

#### Extreme Bedroom values

In [ ]:
house_df['bedrooms'].value_counts()

Theres too little data for houses with nine or more bedrooms (10 out of ~20,000) so we will be excluding it from the training.

In [ ]:
house_df.loc[house_df["bedrooms"] >= 9]

There is only one house with 33 bedrooms. It's too little (1 out of 21436) for the model to have an idea on how to price a house with a feature of this value so we decided to remove it.

In [ ]:
house_df.drop(house_df.loc[house_df["bedrooms"] >= 9].index, inplace=True)
house_df.loc[house_df["bedrooms"] >= 9]

In [ ]:
minMax_df = house_df.drop(columns=columns_to_drop)

min_values = minMax_df.min()
max_values = minMax_df.max()

min_max_df = pd.DataFrame({'min': min_values, 'max': max_values})
min_max_df = min_max_df.round(2)

print(min_max_df)

## Distribution of features

To check for more data that could possibly be suspicious, we check the overall distribution of every feature.

In [ ]:
features = house_df.columns.drop(['id', 'date'])
num_features = len(features)
num_cols = 5
num_rows = (num_features + num_cols - 1) // num_cols

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, num_rows * 4))
axes = axes.flatten()

for i, feature in enumerate(features):
    ax = axes[i]
    sns.histplot(house_df[feature], kde=True, ax=ax)
    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

We don't see other data that we want to remove so we keep it as is.

## Locating outliers in features

We can also check for outliers using boxplots. 

In [ ]:
features = house_df.columns.drop(['id', 'date'])
num_features = len(features)
num_cols = 5
num_rows = (num_features + num_cols - 1) // num_cols

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, num_rows * 4))
axes = axes.flatten()

for i, feature in enumerate(features):
    ax = axes[i]
    sns.boxplot(x=house_df[feature], ax=ax)
    ax.set_title(f'Box Plot of {feature}')
    ax.set_xlabel(feature)

plt.tight_layout()
plt.show()

There's noticeably a lot of outliers for features regarding sqft living but that is expected. <br>
`yr_renovated` has "outliers" because unrenovated houses get a value of 0 for the feature. We leave it as is. 

We make use of Isolation Forest, a model that uses binary trees to find outliers. We use this to check for more outliers that we may have missed. <br>We choose a random state of 42 just to keep the dataset consistent. 

In [ ]:
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outliers = iso_forest.fit_predict(house_df[features])
house_df['outlier'] = outliers
house_df_cleaned = house_df[house_df['outlier'] == 1].drop(columns=['outlier'])
house_df_cleaned.head()

In [ ]:
house_df[house_df['id'] == 192300020]

We display the distribution of the features once again to see how it affected the distribution.

In [ ]:
features = house_df_cleaned.columns.drop(['id', 'date'])
num_features = len(features)
num_cols = 5
num_rows = (num_features + num_cols - 1) // num_cols

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, num_rows * 4))
axes = axes.flatten()

for i, feature in enumerate(features):
    ax = axes[i]
    sns.boxplot(x=house_df_cleaned[feature], ax=ax)
    ax.set_title(f'Box Plot of {feature}')
    ax.set_xlabel(feature)

plt.tight_layout()
plt.show()

## Feature engineering

The age of a buidling can be very important in telling the price of a house. The older it is, the more likely it is to have a lower price. <br>
To get the age of the building, we subtract the year the house was sold with the year the house was built (`year_built`). If the house was renovated, we substitute `yr_built` with `yr_renovated`.

In [ ]:
house_df = house_df_cleaned
house_df['year'] = house_df['date'].dt.year
house_df['age'] = house_df.apply(lambda row: row['year'] - row['yr_renovated'] if row['yr_renovated'] != 0 else row['year'] - row['yr_built'], axis=1)

To check the correctness of the logic, we check for any house that has a negative age. 

In [ ]:
house_df[house_df['age'] < 0]

We realized that some houses may have been bought while they were still being built/renovated, resulting in a negative age. To work around this, we will be giving it an age of 0 as the house since the house is not built yet. 

In [ ]:
house_df['age'] = house_df['age'].replace(-1,0)
pd.DataFrame(house_df['age'].value_counts().sort_index(ascending=True))

With the value count of age being sorted ascendingly, we can see that there are no more rows with a negative value for age. 

In [ ]:
house_df.head()

## Correlation of features

Since we will be attempting to predict the price of a building given these features, we want to see how each feature affects the price.

In [ ]:
correlation_matrix = house_df.corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='YlOrBr', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

So far, the features that positively correlate the most with price are 
- `sqft_living`
- `grade`
- `sqft_above`
- `sqft_living`
- `bathrooms`

## Feature Selection

For now, we will be removing the features below for the following reasons. 
- `id` - the identifier does not really influence the price 
- `yr_built`, `yr_renovated`, `year` and `date` - we are already using `age` so we wouldn't need the features that computationally lead to the new feature we added
- `lat` and `long` - we believe that we can implement this feature a little better so we are removing it for now. 

In [ ]:
col_remove = ['id', 'date', 'yr_built', 'yr_renovated', 'lat', 'long', 'count', 'year']

### ydata-profiling

To get a quick understanding of our data, we use an external library `ydata-profiling`. This library allows us to generate an EDA of the dataset's variables, interactions, correlations and missing values. 

We also specify the correlations we want to see. `phi_k` is great for finding correlations in categorical data while `pearson` is generally great for finding linear relationships between two variables

In [ ]:
profile = ProfileReport(df=house_df,
    title="House Price Profiling Report", 
    correlations={
            "pearson": {"calculate": True},
            "phi_k": {"calculate": True},
        })
profile.to_file('house_prices_ydata_report.html')
profile

Since we've already removed a good amount of data in the previous section, the overal distribution in data look better. We also find a lot less extreme values. 

### When was the most houses sold?

Seeing the time when houses were most sold can help us get a better idea of the data that is included in our data set. 

In [ ]:
house_ym_df = house_df.copy()
house_ym_df['count'] = 1
daily_counts = house_ym_df.groupby('date').count().reset_index()

# Group by month and year, summing the counts
daily_counts['year_month'] = daily_counts['date'].dt.to_period('M')  # Convert to year-month period
monthly_counts = daily_counts.groupby('year_month')['count'].sum().reset_index()

# Convert back to datetime format for plotting
monthly_counts['year_month'] = monthly_counts['year_month'].dt.to_timestamp().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(monthly_counts['year_month'], monthly_counts['count'], marker='o')
plt.ylim(ymin=0)
plt.title("Houses Sold between May 2014 and May 2015")
plt.xlabel("Time")
plt.ylabel("Houses sold")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())  # Show each month

# Rotate date labels
plt.gcf().autofmt_xdate()

plt.show()

The timeline ranges from 2014 to 2015. We can find a slow decline throughout 2014 but a huge increase of houses bought from 2015-01 to 2015-04. A huge dip can be observed again on 2015-05 but this dip can be caused by the data collection period ending during that month. 

In [ ]:
# monthly_counts['year_month'] = monthly_counts['year_month'].dt.to_period('M')
monthly_counts.sort_values(by=['count'], ascending=False)

In [ ]:
monthly_counts['count'].describe()

### What is the distribution of prices of a house overtime?

To get a big picture of the house price distribution throughout time, we put it into a scatter plot, grouped by year-month. 

In [ ]:
house_ym_df['year_month'] = house_ym_df['date'].dt.to_period('M')  # Convert to year-month period

In [ ]:
house_ym_df['price'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
plt.figure(figsize=(45, 6))
plt.scatter(house_ym_df['date'], house_ym_df['price'])

We can notive a few outliers but that's normal as there are naturally going to be only a few expensive houses being sold overtime. 

### What is the distribution of the condition of houses sold?

We do the same but for condition

In [ ]:
house_df['condition'].value_counts()

In [ ]:
bins = [1, 2, 3, 4, 5, 6]
plt.xticks(bins)
plt.title("Condition of Houses Sold")
plt.xlabel("Condition")
plt.ylabel("Frequency")
plt.hist(house_df['condition'], bins=bins, align='left', edgecolor="black", rwidth=0.5)


### What is the distribution of the grade of houses sold?

In [ ]:
house_df['grade'].value_counts()

In [ ]:
bins = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
plt.xticks(bins)
plt.title("Grade of Houses Sold")
plt.xlabel("Grade")
plt.ylabel("Frequency")
plt.hist(house_df['grade'], bins=bins, align='left', edgecolor="black", rwidth=0.5)

### Is sqft_living just sqft_above + sqft_basement?

In [ ]:
# Check if sqft_living equals sqft_above + sqft_basement
mismatch = house_df[house_df['sqft_living'] != house_df['sqft_above'] + house_df['sqft_basement']]

# Index the id where the condition is not met
mismatch_ids = mismatch['id'].tolist()

print("IDs where sqft_living != sqft_above + sqft_basement:", mismatch_ids)

Yes, yes it is just sqft_above + sqft_basement

### Is there a statistically significant difference between condition and grade on price?

In [ ]:
price_grade_condition = house_df[['price', 'grade', 'condition']]
model = ols('price ~ C(grade) + C(condition) + C(grade):C(condition)',
            data=price_grade_condition).fit()
result = sm.stats.anova_lm(model, type=2)
print(result)

### Is there a statistically significant difference between having a waterfront and a good view on price?

In [ ]:
price_waterfront_view = house_df[['price', 'waterfront', 'view']]
model = ols('price ~ C(waterfront) + C(view) + C(waterfront):C(view)',
            data=price_waterfront_view).fit()
result = sm.stats.anova_lm(model, type=2)
print(result)

### Is there a correlation between price and number of bedrooms?

In [ ]:
house_df[['price', 'bedrooms']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Bedrooms and Price of Sold Houses")
plt.xlabel("Number of bedrooms")
plt.ylabel("Price")
plt.scatter(house_df['bedrooms'], house_df['price'], alpha=0.5)

### Is there a correlation between price and number of bathrooms?

In [ ]:
house_df[['price', 'bathrooms']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Bathrooms and Price of Sold Houses")
plt.xlabel("Number of bathrooms")
plt.ylabel("Price")
plt.scatter(house_df['bathrooms'], house_df['price'], alpha=0.5)

### Is there a correlation between price and the size of a house's living area in square feet?

In [ ]:
house_df[['price', 'sqft_living']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Size of Living Area in Square Feet and Price of Sold Houses")
plt.xlabel("Size of Living Area in Square Feet")
plt.ylabel("Price")
plt.scatter(house_df['sqft_living'], house_df['price'], alpha=0.5)

### Is there a correlation between price and the size of a house's lot?

In [ ]:
house_df[['price', 'sqft_lot']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Size of Lot in Square Feet and Price of Sold Houses")
plt.xlabel("Size of Lot in Square Feet")
plt.ylabel("Price")
plt.scatter(house_df['sqft_lot'], house_df['price'], alpha=0.5)

### Is there a correlation between price and total number of floors?

In [ ]:
house_df[['price', 'floors']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Total Number of Floors and Price of Sold Houses")
plt.xlabel("Total Number of Floors")
plt.ylabel("Price")
plt.scatter(house_df['floors'], house_df['price'], alpha=0.5)

### Is there a trend between price and if the property has a waterfront?

In [ ]:
house_df.boxplot("price", by="waterfront", figsize=(5, 5))

### Is there a trend between price and how good of a view the property has?

In [ ]:
house_df.boxplot("price", by="view", figsize=(5, 5))

### Is there a correlation between price and how high the house was built above ground?

In [ ]:
house_df[['price', 'sqft_above']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Square feet above ground and Price of Sold Houses")
plt.xlabel("Square feet above ground")
plt.ylabel("Price")
plt.scatter(house_df['sqft_above'], house_df['price'], alpha=0.5)

### Is there a correlation between price and how low the house was built?

In [ ]:
house_df[['price', 'sqft_basement']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Square Feet below Ground and Price of Sold Houses")
plt.xlabel("Square feet below ground")
plt.ylabel("Price")
plt.scatter(house_df['sqft_basement'], house_df['price'], alpha=0.5)

### Is there a correlation between price and average size of interior housing living space for the closest 15 houses?

In [ ]:
house_df[['price', 'sqft_living15']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Average Size of Interior Housing Living Space for Closest 15 Houses and Price of Sold Houses")
plt.xlabel("Average size of interior housing living space for the closest 15 houses")
plt.ylabel("Price")
plt.scatter(house_df['sqft_living15'], house_df['price'], alpha=0.5)

### Is there a correlation between price and average size of land lots for the closest 15 houses in square feet?

In [ ]:
house_df[['price', 'sqft_lot15']].corr()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Correlation between Average Size of Land Lots for Closest 15 Houses and Price of Sold Houses")
plt.xlabel("Average size of interior housing living space for the land lots for the closest 15 houses")
plt.ylabel("Price")
plt.scatter(house_df['sqft_lot15'], house_df['price'], alpha=0.5)

## Geographical Analysis


To see the overall picture of where the houses are, we plot them out using `folium` and the latitude and longitude provided by the dataset.

### Clustering data

We also try to form clusters in case we notice a trend in the coordinates. 

In [ ]:
X = house_df[['lat', 'long']]

# Calculate WCSS for different number of clusters
wcss = []
for i in range(1, 50):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Plot the elbow graph
plt.plot(range(1, 50, 1), wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

The elbow point should be around 8-10 clusters so we will be using 8 clusters. 

### Mapping clusters to the dataframe

The data is then clustered into 8 groups in total. We display the total amount of clusters below. 

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
house_coor_df = house_df[['id', 'price', 'lat', 'long']].copy()
house_coor_df['cluster'] = kmeans.labels_
cluster_total_house = house_coor_df.groupby('cluster')['id'].count().reset_index()
cluster_total_house

We also display the centers (in lat and long) of the clusters. 

In [ ]:
kmeans.cluster_centers_

The dataset for plotting should now look like this:

In [ ]:
house_coor_df.head()

We include the following features for the geographical analysis: 
- `id` - to identify specific houses
- `price` - to know where the expensive houses are located/check for price density
- `lat` and `long` - for plotting the houses
- `cluster` - to color code the markers for each house. 

### Visualizing Clusters

In [ ]:
# Setting up the map
map_center = [house_coor_df['lat'].mean(), house_coor_df['long'].mean()]
m = folium.Map(location=map_center, zoom_start=9)

# Cluster up points that are close to each other; reduce lag
cluster = MarkerCluster().add_to(m)
color_map = {0: 'red', 1: 'darkgreen', 2: 'lightblue', 3: 'purple', 4: 'beige', 5:'orange', 6:'white', 7:'darkblue'}

# Plotting each house
for i, row in house_coor_df.iterrows():
    color = color_map.get(row['cluster'], 'gray')
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"House ID: {row['id']}<br>Price: {row['price']}<br>Cluster: {row['cluster']}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(cluster)

# Marks to show the center of the clusters within a 10km range
for i, center in enumerate(kmeans.cluster_centers_):
    color = color_map.get(i, 'gray')
    folium.Circle(
        location=[center[0], center[1]],
        radius=10000,
        color="black",
        weight=1,
        fill_opacity=0.2,
        opacity=1,
        fill_color=color,
        fill=False,  # gets overridden by fill_color
        popup="{} meters".format(10000),
        tooltip=f"Cluster: {i}<br>Center: {center[0]}, {center[1]}",
    ).add_to(m)

m

We notice that house `9413400165.0` and `192300020.0` are outliers to the dataset. There are also about 16 houses that are located far away but we will keep them. 

In [ ]:
house_df.drop(house_df.loc[house_df['id'] >= 9413400165].index, inplace=True)
house_df.drop(house_df.loc[house_df['id'] >= 192300020].index, inplace=True)


### Generating a heatmap to show density of house prices


To display the density of houses, we generate a heatmap where waramer colors represent a higher price total within that area. 

In [ ]:
# Setting up the map
map_center = [house_coor_df['lat'].mean(), house_coor_df['long'].mean()]
m = folium.Map(location=map_center, zoom_start=9)

# Cluster up points that are close to each other; reduce lag
cluster = MarkerCluster().add_to(m)
color_map = {0: 'red', 1: 'darkgreen', 2: 'lightblue', 3: 'purple', 4: 'beige', 5:'orange', 6:'white', 7:'darkblue'}

# Plotting each house
for i, row in house_coor_df.iterrows():
    color = color_map.get(row['cluster'], 'gray')
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"House ID: {row['id']}<br>Price: {row['price']}<br>Cluster: {row['cluster']}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(cluster)

# Heat map
heat_data = [[row['lat'], row['long'], row['price']] for i, row in house_df.iterrows()]
HeatMap(heat_data).add_to(m)

m

Some places we found with high price density are:
- Mercer Island
- Aldarra Estates
- White Center

Some places we found with low price density are:
- Vashon Island

# Section 6. Initial model training

### KNN

In [ ]:
knn_df = house_df.drop(columns=col_remove)
knn_df

In [ ]:
X = knn_df.drop('price', axis=1)
y = knn_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
mse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

### Linear

In [ ]:
linear_df = house_df.drop(columns=col_remove)
linear_df

In [ ]:
X = linear_df.drop('price', axis=1)
y = linear_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
mse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

In [ ]:
coefficients = model.coef_

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': coefficients
})

# Sort the DataFrame by the absolute value of the coefficients
feature_importance_df['Absolute Coefficient'] = feature_importance_df['Coefficient'].abs()
feature_importance_df = feature_importance_df.sort_values(by='Absolute Coefficient', ascending=False)

# Display the feature importance
print(feature_importance_df[['Feature', 'Coefficient']])

In [ ]:
ridge = Ridge(alpha=1, solver='auto')
ridge.fit(X_train, y_train)
y_predicted = ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
mse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

In [ ]:
lasso = Lasso(alpha=0.0001, max_iter=100000)
lasso.fit(X_train, y_train)
y_predicted = lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
mse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

### Neural Networks

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes, list_hidden, activation='sigmoid'):
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.num_classes = num_classes
        self.list_hidden = list_hidden
        self.activation = activation
        self.create_network()

    def create_network(self):
        layers = []
        layers.append(nn.Linear(self.input_size, self.list_hidden[0]))
        layers.append(self.get_activation(self.activation))
        for i in range(len(self.list_hidden) - 1):
            layers.append(nn.Linear(self.list_hidden[i], self.list_hidden[i + 1]))
            layers.append(self.get_activation(self.activation))
        layers.append(nn.Linear(self.list_hidden[-1], self.num_classes))
        self.layers = nn.Sequential(*layers)

    def get_activation(self, mode='sigmoid'):
        if mode == 'tanh':
            return nn.Tanh()
        elif mode == 'relu':
            return nn.ReLU(inplace=True)
        return nn.Sigmoid()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
nn_df = house_df.drop(columns=col_remove)
nn_df

In [ ]:
X = nn_df.drop('price', axis=1)
y = nn_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
input_size = X_train.shape[1]
hidden_layers = [64]
output_size = 1
network = NeuralNetwork(input_size, output_size, hidden_layers, activation='relu')

criterion = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

max_epochs = 5
losses = []

for epoch in range(max_epochs):
    current_epoch_loss = 0
    for X, y in zip(X_train.values, y_train.values):
        X = torch.Tensor(X).float()
        y = torch.Tensor([y]).float()

        optimizer.zero_grad()
        outputs = network(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        current_epoch_loss += loss.item()

    average_loss = current_epoch_loss / len(X_train)
    losses.append(average_loss)
    print(f'Epoch: {epoch + 1}, Loss: {average_loss:.6f}')

# Section 7. Error analysis

## Getting the average longitude and latitude from zip codes
zip codes by itself does not express the location of the house well. Longitudes and latitudes would be good but the differences between the coordinate would have differences so small that the models might not see it as significant even if it is.

The model improved a lot in terms of performance when we removed 'zipcode', 'lat' and 'long' so we know those columns are the problem.

We opt to use zip codes like theyre a cluster and use an average latitude and longitude to give the models a numeric idea of the distance each house will have from one another.

### Extracting latitude and longitude out of the zip codes

In [ ]:
zipcode_df = house_df['zipcode'].unique()
zipcode_df = pd.DataFrame(zipcode_df)
zipcode_df = zipcode_df.rename(columns={0:'zipcode'})
zipcode_df

We will be getting the average coordinate of each zipcode using Geopy's geolocator. Since this is open sourced, we have to use a RateLimiter to prevent getting an error from requesting too much. Limiting the rate slows down the data collection so we got the unique zipcode values and queried the coordinates and saved them into a csv file. 

In [ ]:
# # TAKES A WHILE TO RUN!! (a minute on my local machine)
# geolocator = Nominatim(user_agent="geotest")
# geocode = RateLimiter(geolocator.geocode,
#                       min_delay_seconds=1)

# zipcode_df['location'] = zipcode_df['zipcode'].apply(geocode)
# zipcode_df['lat'] = zipcode_df['location'].apply(lambda loc: loc.point.latitude if loc else None)
# zipcode_df['long'] = zipcode_df['location'].apply(lambda loc: loc.point.longitude if loc else None)

# # saving it so that this doesn't have to be run again
# zipcode_df.to_csv("/content/drive/MyDrive/STINTSY_mco/zipcode_df.csv")
# zipcode_df

We read the file from here. 

In [ ]:
zipcode_df = pd.read_csv("zipcode_df.csv")
zipcode_df = zipcode_df.set_index('zipcode')

And add these into a copy of `house_df` called `house_zip_df`. 

In [ ]:
zip_lat_dict = zipcode_df[['lat']].to_dict()
zip_long_dict = zipcode_df[['long']].to_dict()

house_zip_df = house_df.copy()
house_zip_df['zip_lat'] = house_zip_df['zipcode'].map(zip_lat_dict['lat'])
house_zip_df['zip_long'] = house_zip_df['zipcode'].map(zip_long_dict['long'])
house_zip_df

### Plotting zip_lat and zip_long

Now we plot the new dataframe together with `zip_lat` and `zip_long` plotted onto the map to verify the geolocator's accuracy. 

In [ ]:
# Setting up the map
map_center = [house_zip_df['lat'].mean(), house_zip_df['long'].mean()]
m = folium.Map(location=map_center, zoom_start=9)

# Cluster up points that are close to each other; reduce lag
cluster = MarkerCluster().add_to(m)

# Plotting each house
for i, row in house_zip_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"House ID: {row['id']}<br>Price: {row['price']}<br>Zipcode: {row['zipcode']}",
        icon=folium.Icon(icon='info-sign')
    ).add_to(cluster)

# Marks to show the center of the clusters within a 10km range
for i, zipcode in zipcode_df.iterrows():
    color = color_map.get(i, 'gray')
    folium.Circle(
        location=[zipcode['lat'], zipcode['long']],
        radius=3000,
        color="black",
        fill_color='red',
        weight=1,
        fill_opacity=0.2,
        opacity=1,
        fill=False,  # gets overridden by fill_color
        popup="{} meters".format(10000),
        tooltip=f"Cluster: {i}<br>Center: {zipcode['lat']}, { zipcode['long']}",
    ).add_to(m)

m

#### Feasibility

It appears that the zipcodes' latitude and longitude are inaccurate to the point of no longer being in the same continent. The first image shows an overview of where zipcodes ended up on. The next two images are close ups of the inaccurate zipcode locations, one from Italy and another on Lietuva.
<div>
<img src="img/incorrect_zip_coor_summary.png" width="500"/>
</div>
<div>
<img src="img/incorrect_zip_coor_italy.png" width="500"/>
<img src="img/incorrect_zip_coor_lietuva.png" width="500"/>
</div>

#### Conclusion
For this reason, we will continue to use the original latitude and longitude of the houses.

# Section 8. Improving model performance

In [ ]:
house_df.head()

In [ ]:
scaler = MinMaxScaler()
norm_col = house_df.columns.drop(['id', 'date', 'price', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'count', 'year_month', 'year', 'cluster'])
house_df[norm_col] = scaler.fit_transform(house_df[norm_col])

In [ ]:
house_df.head()

## Hyper Parameter Tuning

### KNN

In [ ]:
X = knn_df.drop('price', axis=1)
y = knn_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
k_folds = 5
k_choices = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
scores = np.zeros((len(k_choices), k_folds))

for i in range(len(k_choices)):
    print("k is : " + str(k_choices[i]))
    model = KNeighborsRegressor(n_neighbors=k_choices[i])
    scores[i] = cross_val_score(model, X_train, y_train, cv=k_folds)
    pass

avg_scores = np.mean(scores, axis=1)
avg_scores

In [ ]:
print("Best k: ", k_choices[np.argmax(avg_scores)])

### Linear

In [ ]:
model = Ridge()
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='r2')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

In [ ]:
kfold = KFold(n_splits=5)
stratified_kfold = StratifiedKFold(n_splits=5)
time_series_split = TimeSeriesSplit(n_splits=5)

# Ridge Regression
ridge = Ridge()
ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}
ridge_grid_search = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, cv=5, scoring='r2')
ridge_grid_search.fit(X_train, y_train)
print(f"Best Ridge parameters kfold: {ridge_grid_search.best_params_}")
print(f"Best Ridge cross-validation score kfold: {ridge_grid_search.best_score_}")

ridge_grid_search_kfold = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, cv=kfold, scoring='r2')
ridge_grid_search_kfold.fit(X_train, y_train)
print(f"Best Ridge parameters kfold: {ridge_grid_search_kfold.best_params_}")
print(f"Best Ridge cross-validation score kfold: {ridge_grid_search_kfold.best_score_}")

ridge_grid_search_stratified = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, cv=stratified_kfold, scoring='r2')
ridge_grid_search_stratified.fit(X_train, y_train)
print(f"Best Ridge parameters stratified: {ridge_grid_search_stratified.best_params_}")
print(f"Best Ridge cross-validation score stratified: {ridge_grid_search_stratified.best_score_}")

ridge_grid_search_time_series = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, cv=time_series_split, scoring='r2')
ridge_grid_search_time_series.fit(X_train, y_train)
print(f"Best Ridge parameters time series: {ridge_grid_search_time_series.best_params_}")
print(f"Best Ridge cross-validation score time series: {ridge_grid_search_time_series.best_score_}")

# Lasso Regression
lasso = Lasso()
lasso_param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'max_iter': [1000, 10000, 100000]
}
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=5, scoring='r2')
lasso_grid_search.fit(X_train, y_train)
print(f"Best Lasso parameters: {lasso_grid_search.best_params_}")
print(f"Best Lasso cross-validation score: {lasso_grid_search.best_score_}")

lasso_grid_search_kfold = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=kfold, scoring='r2')
lasso_grid_search_kfold.fit(X_train, y_train)
print(f"Best Lasso parameters kfold: {lasso_grid_search_kfold.best_params_}")
print(f"Best Lasso cross-validation score kfold: {lasso_grid_search_kfold.best_score_}")

lasso_grid_search_stratified = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=stratified_kfold, scoring='r2')
lasso_grid_search_stratified.fit(X_train, y_train)
print(f"Best Lasso parameters stratified: {lasso_grid_search_stratified.best_params_}")
print(f"Best Lasso cross-validation score stratified: {lasso_grid_search_stratified.best_score_}")

lasso_grid_search_time_series = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=time_series_split, scoring='r2')
lasso_grid_search_time_series.fit(X_train, y_train)
print(f"Best Lasso parameters time series: {lasso_grid_search_time_series.best_params_}")
print(f"Best Lasso cross-validation score time series: {lasso_grid_search_time_series.best_score_}")

### Neural Networks

In [ ]:
def create_model(optimizer='adam', init='uniform', learning_rate=0.01, hidden_layers=[64, 32]):
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=input_size, kernel_initializer=init, activation='relu'))
    for units in hidden_layers[1:]:
        model.add(Dense(units, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init))
    model.compile(loss='mean_squared_error', optimizer=optimizer, learning_rate=learning_rate)
    return model

# Wrap the model using KerasRegressor
model = KerasRegressor(build_fn=create_model, verbose=0)

# Define the grid of hyperparameters
param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [50, 100, 200],
    'optimizer': ['SGD', 'Adam'],
    'init': ['uniform', 'normal'],
    'learning_rate': [0.001, 0.01, 0.1],
    'hidden_layers': [[64], [64, 32], [128, 64, 32]]
}

# Set up the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fit the grid search
grid_result = grid.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_result.best_params_
best_score = grid_result.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

### KNN

In [ ]:
col_remove = ['id', 'date', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'count', 'year_month', 'year', 'cluster']
knn_df = house_df.drop(columns=col_remove)
knn_df

In [ ]:
X = knn_df.drop('price', axis=1)
y = knn_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
model = KNeighborsRegressor(n_neighbors=8)
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
mse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

In [ ]:
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_predicted})
print(results_df)

### Linear


In [ ]:
linear_df = house_df.drop(columns=col_remove)
linear_df

In [ ]:
X = linear_df.drop('price', axis=1)
y = linear_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(max_iter=1000, tol=1e-3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted House Prices')
plt.show()

In [ ]:
coefficients = model.coef_

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': coefficients
})

# Sort the DataFrame by the absolute value of the coefficients
feature_importance_df['Absolute Coefficient'] = feature_importance_df['Coefficient'].abs()
feature_importance_df = feature_importance_df.sort_values(by='Absolute Coefficient', ascending=False)

# Display the feature importance
print(feature_importance_df[['Feature', 'Coefficient']])

In [ ]:
coefficients = model.coef_
features = X.columns

# Create a DataFrame for the coefficients
coef_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})

# Sort the DataFrame by the absolute value of the coefficients
coef_df['Absolute Coefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='Absolute Coefficient', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(coef_df['Feature'], coef_df['Coefficient'], color='skyblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Feature Importance (Coefficients) in Linear Regression Model')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest coefficient at the top
plt.show()

In [ ]:
ridge = Ridge(alpha=1, solver='auto')
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

In [ ]:
lasso = Lasso(alpha=0.0001, max_iter=100000)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mae: {mae}")
print(f"mse: {mse}")
print(f"r2: {r2}")

### Neural Networks

In [ ]:
nn_df = house_df.drop(columns=col_remove)
nn_df

In [ ]:
X = nn_df.drop('price', axis=1)
y = nn_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
input_size = X_train.shape[1]
hidden_layers = [64]
output_size = 1
network = NeuralNetwork(input_size, output_size, hidden_layers, activation='relu')

criterion = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

max_epochs = 300
losses = []

for epoch in range(max_epochs):
    current_epoch_loss = 0
    for X, y in zip(X_train.values, y_train.values):
        X = torch.Tensor(X).float()
        y = torch.Tensor([y]).float()

        optimizer.zero_grad()
        outputs = network(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        current_epoch_loss += loss.item()

    average_loss = current_epoch_loss / len(X_train)
    losses.append(average_loss)
    print(f'Epoch: {epoch + 1}, Loss: {average_loss:.6f}')

In [ ]:
network.eval()
with torch.no_grad():
    y_pred = network(torch.Tensor(X_test.values).float()).numpy()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Section 9. Model performance summary

# Section 10. Insights and conclusions

# Section 11. References